In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
import groq

# Initialize Groq API client
client = groq.Client(api_key="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")

In [ ]:
# Function to demonstrate Chain of Thought (CoT)
def chain_of_thought_example():
    prompt = """
    Let's solve this math problem step by step:
    If a train travels at 80 km/h for 3 hours, how far does it travel?
    """
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "system", "content": "You are a helpful assistant that follows the Chain of Thought process."},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [ ]:
# Function to demonstrate ReAct (Reasoning + Acting)
def react_example():
    prompt = """
    You are an AI agent that first reasons, then acts. Answer this question step by step:
    Who won the FIFA World Cup in 2022?
    """
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "system", "content": "You are an AI agent using the ReAct approach."},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [ ]:
# Function to demonstrate Reflection
def reflection_example():
    initial_prompt = "Write a short story about a brave cat."
    initial_response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "system", "content": "You are a creative storyteller."},
                  {"role": "user", "content": initial_prompt}]
    )
    story = initial_response.choices[0].message.content

    # AI Reflecting on its own story and improving it
    reflection_prompt = f"Improve the following story by adding more depth and creativity: {story}"
    improved_response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "system", "content": "You are an AI that reflects on previous responses and improves them."},
                  {"role": "user", "content": reflection_prompt}]
    )
    return improved_response.choices[0].message.content

# Additional function to test various aspects
def test_all_concepts():
    print("\n=== Chain of Thought Example ===")
    print(chain_of_thought_example())

    print("\n=== ReAct Example ===")
    print(react_example())

    print("\n=== Reflection Example ===")
    print(reflection_example())

    print("\n=== Testing Multi-Step Reasoning ===")
    multi_step_prompt = """
    A farmer has 10 apples. He gives away 3 apples to his friend and then buys 5 more apples.
    How many apples does he have now? Solve this step by step.
    """
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "system", "content": "Solve the problem step by step using logical reasoning."},
                  {"role": "user", "content": multi_step_prompt}]
    )
    print(response.choices[0].message.content)

    print("\n=== Testing Decision Making and Planning ===")
    planning_prompt = """
    Imagine you are an AI assistant helping a user plan a one-week trip to Japan.
    Provide a detailed itinerary including places to visit, food recommendations, and estimated travel costs.
    """
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "system", "content": "You are an expert travel planner."},
                  {"role": "user", "content": planning_prompt}]
    )
    print(response.choices[0].message.content)

# Running and validating outputs
if __name__ == "__main__":
    test_all_concepts()



=== Chain of Thought Example ===
Sounds like a fun problem!

To solve this problem, I'll follow the Chain of Thought process. Here's my thought process:

1. The problem gives us the train's speed: 80 km/h.
2. The problem also gives us the time the train travels: 3 hours.
3. We need to find out how far the train travels.

To find the distance traveled, I'll multiply the speed (80 km/h) by the time (3 hours).

4. I'll multiply 80 km/h by 3 hours:

80 km/h × 3 hours = ?

Now, let me calculate...

5. The result of the multiplication is:

80 km/h × 3 hours = 240 km

So, the train travels 240 km in 3 hours!

The answer is: 240 km

=== ReAct Example ===
A question that requires reasoning and then acting!

**Reasoning:**
To answer this question, I need to use my knowledge database and reasoning capabilities. I will search for information related to the FIFA World Cup in 2022 and try to find the answer.

**Fact-finding:**
After searching my database, I found that the FIFA World Cup in 2022 was

In [ ]:
import re
import time

# Memory to store tasks (our "state")
task_list = {}

# Intent detection function
def detect_intent(user_input):
    user_input = user_input.lower().strip()

    # Define simple intent rules using regex and keywords
    if re.search(r'add|create|new', user_input):
        return "add_task"
    elif re.search(r'list|show|view', user_input):
        return "list_tasks"
    elif re.search(r'remove|delete|clear', user_input):
        return "remove_task"
    else:
        return "unknown"

# Entity extraction function
def extract_entities(user_input):
    user_input = user_input.lower().strip()
    entities = {}

    # Extract task name (simple: assume it's after "add" or "task")
    task_match = re.search(r'(add|task)\s+(.+?)(?:\s+at|\s+on|$)', user_input)
    if task_match:
        entities['task_name'] = task_match.group(2).strip()

    # Extract time (e.g., "at 3pm" or "on Monday")
    time_match = re.search(r'(at|on)\s+(.+?)(?:\s+|$)', user_input)
    if time_match:
        entities['time'] = time_match.group(2).strip()

    return entities

# Action handlers
def add_task(entities):
    if 'task_name' not in entities:
        return "Sorry, I didn’t catch the task name. What would you like to add?"

    task_name = entities['task_name']
    task_id = str(time.time())  # Unique ID based on timestamp
    task_list[task_id] = {
        'name': task_name,
        'time': entities.get('time', 'unspecified')
    }
    return f"Added task: '{task_name}' (Time: {task_list[task_id]['time']})"

def list_tasks():
    if not task_list:
        return "You don’t have any tasks yet!"

    response = "Your tasks:\n"
    for task_id, task in task_list.items():
        response += f"- {task['name']} (Time: {task['time']})\n"
    return response.strip()

def remove_task(entities):
    if 'task_name' not in entities:
        return "Which task would you like to remove?"

    task_name = entities['task_name']
    for task_id, task in list(task_list.items()):
        if task['name'].lower() == task_name:
            del task_list[task_id]
            return f"Removed task: '{task_name}'"
    return f"Couldn’t find task: '{task_name}'"

# Response generation
def generate_response(intent, entities):
    if intent == "add_task":
        return add_task(entities)
    elif intent == "list_tasks":
        return list_tasks()
    elif intent == "remove_task":
        return remove_task(entities)
    else:
        return "Sorry, I didn’t understand that. I can add, list, or remove tasks. How can I help?"

# Main agent loop
def run_agent():
    print("Hello! I’m your Task Assistant. How can I help you today?")
    while True:
        user_input = input("> ")
        if user_input.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break

        # Process input
        intent = detect_intent(user_input)
        entities = extract_entities(user_input)
        response = generate_response(intent, entities)
        print(response)

# Start the agent
if __name__ == "__main__":
    run_agent()

Hello! I’m your Task Assistant. How can I help you today?
> hi
Sorry, I didn’t understand that. I can add, list, or remove tasks. How can I help?
> hi tehre
Sorry, I didn’t understand that. I can add, list, or remove tasks. How can I help?


KeyboardInterrupt: Interrupted by user

In [ ]:
import torch
import tensorflow as tf
import os

# 1️⃣ Check GPU availability
gpu_available = torch.cuda.is_available()
print(f"✅ GPU Available: {gpu_available}")

# 2️⃣ Check CUDA version
cuda_version = torch.version.cuda
print(f"✅ CUDA Version: {cuda_version}")

# 3️⃣ Check cuDNN version
cudnn_version = torch.backends.cudnn.version()
print(f"✅ cuDNN Version: {cudnn_version}")

# 4️⃣ Check TensorFlow GPU support
print(f"✅ TensorFlow GPU Available: {tf.config.list_physical_devices('GPU')}")

# 5️⃣ Run NVIDIA-SMI
!nvidia-smi

# 6️⃣ Check environment variables (CUDA paths)
print(f"✅ CUDA Path: {os.environ.get('CUDA_PATH')}")
print(f"✅ LD_LIBRARY_PATH: {os.environ.get('LD_LIBRARY_PATH')}")



✅ GPU Available: True
✅ CUDA Version: 12.4
✅ cuDNN Version: 90300
✅ TensorFlow GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Tue Mar 11 10:08:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       2MiB /  15360MiB |   

In [ ]:

## 5️⃣ Test GPU Performance Using PyTorch (Matrix Multiplication)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
a = torch.randn(1000, 1000, device=device)
b = torch.randn(1000, 1000, device=device)
%time c = torch.matmul(a, b)


CPU times: user 34.1 ms, sys: 14.2 ms, total: 48.3 ms
Wall time: 202 ms


In [ ]:
## 6️⃣ Test GPU Performance Using TensorFlow (Matrix Multiplication)
with tf.device('/GPU:0'):
    A = tf.random.normal([1000, 1000])
    B = tf.random.normal([1000, 1000])
    %time C = tf.matmul(A, B)

CPU times: user 2.07 ms, sys: 12 µs, total: 2.08 ms
Wall time: 2.26 ms


In [ ]:
## 7️⃣ Interfacing with Groq API for AI Processing
import requests
import json

API_URL = "https://api.groq.com/v1/chat/completions"
HEADERS = {"Authorization": "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", "Content-Type": "application/json"}

def chat_with_groq(prompt):
    payload = {
        "model": "llama3-8b-8192",
        "messages": [{"role": "user", "content": prompt}],
    }
    response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))
    return response.json()

# Example Query to Groq API
response = chat_with_groq("Explain how CUDA accelerates deep learning models.")
print(response)


{'error': {'message': 'Unknown request URL: POST /v1/chat/completions. Please check the URL for typos, or see the docs at https://console.groq.com/docs/', 'type': 'invalid_request_error', 'code': 'unknown_url'}}


In [ ]:

## 5️⃣ GPU Performance Test - Large-Scale Matrix Multiplication
# Testing GPU with Large Tensor Computation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
a = torch.randn(5000, 5000, device=device)
b = torch.randn(5000, 5000, device=device)
%time c = torch.matmul(a, b)

CPU times: user 0 ns, sys: 541 µs, total: 541 µs
Wall time: 563 µs


In [ ]:
## 6️⃣ AI Use Case - Deep Learning Model Training on GPU
# Training a simple CNN model on CIFAR-10 dataset using PyTorch
def train_cifar10():
    import torch.nn as nn
    import torch.optim as optim
    import torchvision
    import torchvision.transforms as transforms

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 32, 3, 1)  # Output: (32, 30, 30)
            self.pool = nn.MaxPool2d(2, 2)       # Output: (32, 15, 15)
            self.conv2 = nn.Conv2d(32, 64, 3, 1) # Output: (64, 13, 13)
            self.pool2 = nn.MaxPool2d(2, 2)      # Output: (64, 6, 6)
            self.fc1 = nn.Linear(64 * 6 * 6, 128)
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = torch.relu(self.conv1(x))
            x = self.pool(x)
            x = torch.relu(self.conv2(x))
            x = self.pool2(x)
            x = torch.flatten(x, 1)  # Flatten before FC layers
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    net = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    for epoch in range(1):  # Single epoch training for quick testing
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                print(f'Epoch [{epoch+1}], Step [{i}], Loss: {loss.item():.4f}')

train_cifar10()

Files already downloaded and verified
Epoch [1], Step [0], Loss: 2.2964
Epoch [1], Step [100], Loss: 1.5724
Epoch [1], Step [200], Loss: 1.5463
Epoch [1], Step [300], Loss: 1.5733
Epoch [1], Step [400], Loss: 1.2153
Epoch [1], Step [500], Loss: 1.1393
Epoch [1], Step [600], Loss: 1.2246
Epoch [1], Step [700], Loss: 1.3104


In [ ]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 576 MiB, 14518 MiB


In [ ]:
!nvidia-smi pmon -i 0 -s um -c 10


    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              
    0          -     -      -      -      -      -      -      -      -      -    -              


In [ ]:
!nvidia-smi --query-gpu=power.draw,power.limit,temperature.gpu --format=csv


power.draw [W], power.limit [W], temperature.gpu
27.16 W, 70.00 W, 46


In [ ]:
# Google Colab Notebook: NVIDIA Drivers, CUDA, cuDNN, and NVIDIA-SMI Analysis with AI Workloads

## 1️⃣ Check NVIDIA GPU Availability and Specifications
!nvidia-smi

## 2️⃣ Verify CUDA Installation and Version
import torch
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
print(f"✅ CUDA Version: {torch.version.cuda}")

## 3️⃣ Check cuDNN Installation
print(f"✅ cuDNN Version: {torch.backends.cudnn.version()}")

## 4️⃣ Verify TensorFlow GPU Support
import tensorflow as tf
print(f"✅ TensorFlow GPU: {tf.config.list_physical_devices('GPU')}")

## 5️⃣ GPU Performance Test - Large-Scale Matrix Multiplication
# Testing GPU with Large Tensor Computation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
a = torch.randn(5000, 5000, device=device)
b = torch.randn(5000, 5000, device=device)
%time c = torch.matmul(a, b)

## 6️⃣ AI Use Case - Deep Learning Model Training on GPU
# Training a simple CNN model on CIFAR-10 dataset using PyTorch
def train_cifar10():
    import torch.nn as nn
    import torch.optim as optim
    import torchvision
    import torchvision.transforms as transforms

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 32, 3, 1)  # Output: (32, 30, 30)
            self.pool = nn.MaxPool2d(2, 2)       # Output: (32, 15, 15)
            self.conv2 = nn.Conv2d(32, 64, 3, 1) # Output: (64, 13, 13)
            self.pool2 = nn.MaxPool2d(2, 2)      # Output: (64, 6, 6)
            self.fc1 = nn.Linear(64 * 6 * 6, 128)
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = torch.relu(self.conv1(x))
            x = self.pool(x)
            x = torch.relu(self.conv2(x))
            x = self.pool2(x)
            x = torch.flatten(x, 1)  # Flatten before FC layers
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    net = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    for epoch in range(1):  # Single epoch training for quick testing
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                print(f'Epoch [{epoch+1}], Step [{i}], Loss: {loss.item():.4f}')

train_cifar10()

## 7️⃣ NVIDIA-SMI Utilization Monitoring
## 8️⃣ Process-Specific GPU Monitoring
!nvidia-smi pmon -i 0 -s um -c 10

## 9️⃣ Power and Temperature Profiling
!nvidia-smi --query-gpu=power.draw,power.limit,temperature.gpu --format=csv

## 🔟 Run AI Workload & Observe Real-time Utilization
x = torch.randn(10000, 10000, device=device)
y = torch.randn(10000, 10000, device=device)
print("Performing large matrix multiplication on GPU...")
%time z = torch.matmul(x, y)

## 1️⃣1️⃣ Interfacing with Groq API for AI Processing
import requests
import json

API_URL = "https://api.groq.com/v1/chat/completions"
HEADERS = {"Authorization": "Bearer YOUR_GROQ_API_KEY", "Content-Type": "application/json"}

def chat_with_groq(prompt):
    payload = {
        "model": "llama3-8b-8192",
        "messages": [{"role": "user", "content": prompt}],
    }
    response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))
    return response.json()

# Example Query to Groq API
response = chat_with_groq("Explain how CUDA accelerates deep learning models.")
print(response)


Tue Mar 11 11:08:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----